In [2]:
from lxml import etree as ET
import shutil
from subprocess import check_output
from pathlib import Path
import numpy as np
import os
#import matplotlib.pyplot as plt
import pandas as pd
#from shapely.geometry import Polygon,
import RClib as RClib



In [3]:
%load_ext autoreload
%autoreload 2
''
config = RClib.loadconfigs('.\config.json')
scandf = RClib.provide_scandf(inputdirectory=config['workspace'], imageformat = '*.dng')
scandf= scandf.apply(RClib.defineRawTherapeeOutput, foldername='02_PNG', axis=1)
scandf= scandf.apply(RClib.defineRealityCaptureOutput, foldername='03_RC', axis=1)
rcsettings_template = pd.read_csv('.\\allRCsettings_template.csv', sep=';', header=0) 
#print(rcsettings_template)



In [8]:

scandf2=pd.DataFrame()
for index,scan in scandf.iterrows():
    scan['imagedf'] = scan['imagedf'].apply(RClib.developwithRawTherapee, pp3filepath=scan['pp3file'][0], outputfolderpath=scan['RToutputfolder'], axis=1) 
    scan = RClib.makeImagelist(scan, imagelistname=scan['id'])
    scan = RClib.createRCproject(scan)
    scan['commandlist_alignsettings']=Path(config['alignmentsettings'])
    print(scan['commandlist_alignsettings'])
    RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_alignsettings' )
    scan['commandlist_align'] = []
    scan['commandlist_align'].append('-add ' + str(scan['list_dev-img_path']))
    scan['commandlist_align'].append('-detectMarkers')
    scan['commandlist_align'].append('-align')
    scan['commandlist_align'].append('-save')
    scan['commandlist_align'].append('-quit')
    scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_align', rccmdpathfield='align-rccmdpath')
    scan = RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='align-rccmdpath' )
    scandf2 = scandf2.append(scan)
pd.write_csv(Path(config['workspace']) / 'rcprocessingdf.csv', sep=';')


E:\GitHub\RCprocessing\RCsettings_alignment.rccmd
E:\RCprocessing_test\PG_boat91_test\03_RC\commandlist_align.rccmd
E:\GitHub\RCprocessing\RCsettings_alignment.rccmd
E:\RCprocessing_test\PG_boat_test\03_RC\commandlist_align.rccmd


AttributeError: module 'pandas' has no attribute 'write_csv'